In [4]:
import os
import boto3
import time
import re
from sagemaker import get_execution_role

role = get_execution_role()

# Now let's define the S3 bucket we'll used for the remainder of this example.

bucket = 'instaanalytics' #  enter your s3 bucket where you will copy data and model artificats

In [5]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import sklearn as sk                              # For access to a variety of machine learning models
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from sklearn.datasets import dump_svmlight_file   # For outputting data to libsvm format for xgboost
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting output
import io                                         # For working with stream data
import sagemaker.amazon.common as smac            # For protobuf data format

In [6]:
def get_matching_s3_keys(bucket, prefix='', suffix=''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    s3 = boto3.client('s3')
    kwargs = {'Bucket': bucket}

    # If the prefix is a single string (not a tuple of strings), we can
    # do the filtering directly in the S3 API.
    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:

        # The S3 API response is a large blob of metadata.
        # 'Contents' contains information about the listed objects.
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix) and key.endswith(suffix):
                yield key

        # The S3 API is paginated, returning up to 1000 keys at a time.
        # Pass the continuation token into the next response, until we
        # reach the final page (when this field is missing).
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [10]:
prefix = 'bad'
suffix = '.json'
df_list = []
for key in get_matching_s3_keys(bucket=bucket, prefix=prefix, suffix=suffix):
    temp_df = pd.read_json('s3://{}/{}'.format(bucket,key))
    print(temp_df.shape)
    df_list.append(temp_df)

(3, 4)
(2, 4)
(17, 4)
(2, 4)
(1, 4)
(1, 4)
(1, 4)
(2, 4)
(5, 4)
(10, 4)
(2, 4)
(4, 4)
(3, 4)
(2, 4)
(4, 4)
(2, 4)
(2, 4)
(2, 4)
(2, 4)
(2, 4)
(9, 4)
(5, 4)
(3, 4)
(4, 4)
(2, 4)
(2, 4)
(3, 4)
(2, 4)
(4, 4)
(1, 4)
(4, 4)
(4, 4)
(1, 4)
(5, 4)
(6, 4)
(2, 4)
(4, 4)
(6, 4)
(6, 4)
(9, 4)
(2, 4)
(10, 4)
(2, 4)
(2, 4)
(2, 4)
(2, 4)
(4, 4)
(1, 4)
(1, 4)
(3, 4)
(3, 4)
(8, 4)
(5, 4)
(7, 4)
(5, 4)
(1, 4)
(2, 4)
(8, 4)
(5, 4)
(9, 4)
(1, 4)
(7, 4)
(4, 4)
(3, 4)
(1, 4)
(1, 4)
(4, 4)
(3, 4)
(1, 4)
(2, 4)
(21, 4)
(1, 4)
(3, 4)
(2, 4)
(1, 4)
(1, 4)
(2, 4)
(4, 4)
(2, 4)
(3, 4)
(2, 4)
(3, 4)
(8, 4)
(1, 4)
(1, 4)
(1, 4)
(7, 4)
(10, 4)
(1, 4)
(4, 4)
(3, 4)
(1, 4)
(2, 4)
(2, 4)
(1, 4)
(4, 4)
(1, 4)
(1, 4)
(2, 4)
(2, 4)
(1, 4)
(2, 4)
(2, 4)
(1, 4)
(3, 4)
(1, 4)
(6, 4)
(1, 4)
(3, 4)
(3, 4)
(2, 4)
(1, 4)
(2, 4)
(13, 4)
(1, 4)
(8, 4)
(1, 4)
(2, 4)
(14, 4)
(2, 4)
(1, 4)
(3, 4)
(1, 4)
(2, 4)
(1, 4)
(5, 4)
(4, 4)
(7, 4)
(2, 4)
(3, 4)
(20, 4)


KeyboardInterrupt: 